### 1. <a id='importation'>Détail des données fournies</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings

# Ignorer les avertissements
warnings.filterwarnings('ignore')
path = "../data/"

DELIMITER = "--------------------------------"

---

### 2. <a id='chargement'>Chargement et initialisation des données</a>

In [5]:
#On vient charger les différents dataset
employee_df = pd.read_csv(path + "employee_survey_data.csv")
general_df = pd.read_csv(path + "general_data.csv")
manager_df = pd.read_csv(path + "manager_survey_data.csv")
in_time_df = pd.read_csv(path + "in_time.csv")
out_time_df = pd.read_csv(path + "out_time.csv")

dataframes = [
    ("employee_df", employee_df),
    ("general_df", general_df),
    ("manager_df", manager_df),
    ("in_time_df", in_time_df),
    ("out_time_df", out_time_df)
]

for name, df in dataframes:
    print(f"Nom du DataFrame : {name}")
    print(f"Description : {df.describe()}")
    print(f" Valeurs nulles : {df.isnull().sum()}")


# result employee_survey = pd.read_csv('../data/cleaned_employee_survey.csv')
#general_data = pd.read_csv('../data/cleaned_general_data.csv')
#manager_survey = pd.read_csv('../data/cleaned_manager_survey.csv')
#total_hours_per_employee = pd.read_csv('../data/total_hours_per_employee.csv')

Nom du DataFrame : employee_df
Description :         EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance
count  4410.000000              4385.000000      4390.000000      4372.000000
mean   2205.500000                 2.723603         2.728246         2.761436
std    1273.201673                 1.092756         1.101253         0.706245
min       1.000000                 1.000000         1.000000         1.000000
25%    1103.250000                 2.000000         2.000000         2.000000
50%    2205.500000                 3.000000         3.000000         3.000000
75%    3307.750000                 4.000000         4.000000         3.000000
max    4410.000000                 4.000000         4.000000         4.000000
 Valeurs nulles : EmployeeID                  0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64
Nom du DataFrame : general_df
Description :                Age  DistanceFromHome    Education  Employe

---

Résumé des différentes données présentes dans les dataset:
(ce qu'on a fait dans chaque notebook)


---


### 3. <a id='merge'>Pré-traitements spécifique</a>

Notre objectif est de venir merger ces différents dataset afin de travailler avec un seul DataFrame.
En observant les différentes caractéristiques des dataset, on s'est aperçu qu'on peut dès à présent fusionner tous les dataset sauf "in_time" et "out_time" car les informations présentes sont très différentes des autres dataset, sur la forme ou le fond.
On va donc venir synthétiser ces données afin de récupérer l'heure totale de travail pour chaque employé.


In [11]:
in_time_df = in_time_df.rename(columns={"Unnamed: 0": "EmployeeID"})
out_time_df = out_time_df.rename(columns={"Unnamed: 0": "EmployeeID"})

### Vérifications concernant les 2 dataset :

# Valeurs des EmployeeID
if not in_time_df['EmployeeID'].equals(out_time_df['EmployeeID']):
    print("Les EmployeeID ne correspondent pas entre les dataset : in_time_df et out_time_df")

# Correspondance des valeurs NA
in_time_na = in_time_df.isna()
out_time_na = out_time_df.isna()
if not in_time_na.equals(out_time_na):
    print("Les valeurs NA dans in_time_df et out_time_df ne correspondent pas")

### On est maintenant sûr que les dataset sont conformes et peuvent donc être fusionnés, on va venir convertir les objets en date afin de récuperer les heures totales.

for col in in_time_df.columns[1:]:
    in_time_df[col] = pd.to_datetime(in_time_df[col], errors='coerce')
    out_time_df[col] = pd.to_datetime(out_time_df[col], errors='coerce')

time_difference = out_time_df.iloc[:, 1:] - in_time_df.iloc[:, 1:]

time_difference_in_hours = time_difference.apply(lambda row: row.dt.total_seconds() / 3600, axis=1)
total_hours_per_employee = time_difference_in_hours.sum(axis=1, skipna=True).round(2)

# Ajouter une colonne pour la durée totale
out_time_df["Total_Hours"] = total_hours_per_employee
total_hours_df = out_time_df[["EmployeeID", "Total_Hours"]]

,EmployeeID,Total_Hours
0,1,1710.69
1,2,1821.68
2,3,1697.20
3,4,1690.51
4,5,1961.51
...,...,...
4405,4406,2070.91
4406,4407,1468.40
4407,4408,1780.23
4408,4409,2287.72


### 3. <a id='merge'>Merge et sauvegarde des datasets</a>

In [12]:
company_df = general_df.merge(employee_df, on='EmployeeID').merge(manager_df, on='EmployeeID').merge(total_hours_df, on='EmployeeID')
company_df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Total_Hours
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,6,1,0,0,3.0,4.0,2.0,3,3,1710.69
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,3,5,1,4,3.0,2.0,4.0,2,4,1821.68
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,2,5,0,3,2.0,2.0,1.0,3,3,1697.20
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,5,8,7,5,4.0,4.0,3.0,2,3,1690.51
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,2,6,0,4,4.0,1.0,3.0,3,3,1961.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,...,5,3,0,2,4.0,1.0,3.0,3,3,2070.91
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,...,2,3,0,2,4.0,4.0,3.0,2,3,1468.40
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,...,4,4,1,2,1.0,3.0,3.0,3,4,1780.23
4408,42,No,Travel_Rarely,Sales,18,2,Medical,1,4409,Male,...,2,9,7,8,4.0,1.0,3.0,2,3,2287.72


---

### 4. <a id='apercu'>Aperçu général</a>

In [18]:
print(f"Nombre de lignes : {company_df.shape[0]}")
print(f"Nombre de colonnes : {company_df.shape[1]}")

Nombre de lignes : 4410
Nombre de colonnes : 48


In [19]:
print("Informations de base :")
company_df.info()

Informations de base :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                4410 non-null   int64  
 1   Attrition                          4410 non-null   int64  
 2   DistanceFromHome                   4410 non-null   int64  
 3   Education                          4410 non-null   int64  
 4   EmployeeCount                      4410 non-null   int64  
 5   EmployeeID                         4410 non-null   int64  
 6   JobLevel                           4410 non-null   int64  
 7   MonthlyIncome                      4410 non-null   int64  
 8   NumCompaniesWorked                 4410 non-null   float64
 9   Over18                             4410 non-null   int64  
 10  PercentSalaryHike                  4410 non-null   int64  
 11  StandardHours                    

In [20]:
print(f"Sommaire des statistiques : \n{company_df.describe()}")

Sommaire des statistiques : 
               Age    Attrition  DistanceFromHome    Education  EmployeeCount  \
count  4410.000000  4410.000000       4410.000000  4410.000000         4410.0   
mean     36.923810     0.161224          9.192517     1.912925            1.0   
std       9.133301     0.367780          8.105026     1.023933            0.0   
min      18.000000     0.000000          1.000000     0.000000            1.0   
25%      30.000000     0.000000          2.000000     1.000000            1.0   
50%      36.000000     0.000000          7.000000     2.000000            1.0   
75%      43.000000     0.000000         14.000000     3.000000            1.0   
max      60.000000     1.000000         29.000000     4.000000            1.0   

        EmployeeID     JobLevel  MonthlyIncome  NumCompaniesWorked  Over18  \
count  4410.000000  4410.000000    4410.000000         4410.000000  4410.0   
mean   2205.500000     2.063946   65029.312925            2.691837     1.0   
std    

In [21]:
print("Nombre de valeurs uniques par colonne :")
for column in company_df.columns:
    print(f"- {column} : {company_df[column].nunique()} valeurs uniques")

Nombre de valeurs uniques par colonne :
- Age : 43 valeurs uniques
- Attrition : 2 valeurs uniques
- DistanceFromHome : 29 valeurs uniques
- Education : 5 valeurs uniques
- EmployeeCount : 1 valeurs uniques
- EmployeeID : 4410 valeurs uniques
- JobLevel : 5 valeurs uniques
- MonthlyIncome : 1349 valeurs uniques
- NumCompaniesWorked : 10 valeurs uniques
- Over18 : 1 valeurs uniques
- PercentSalaryHike : 15 valeurs uniques
- StandardHours : 1 valeurs uniques
- StockOptionLevel : 4 valeurs uniques
- TotalWorkingYears : 40 valeurs uniques
- TrainingTimesLastYear : 7 valeurs uniques
- YearsAtCompany : 37 valeurs uniques
- YearsSinceLastPromotion : 16 valeurs uniques
- YearsWithCurrManager : 18 valeurs uniques
- AvgYearsPerCompany : 167 valeurs uniques
- StabilityUnderManager : 117 valeurs uniques
- TravelIntensity : 45 valeurs uniques
- YearsBeforePromotion : 31 valeurs uniques
- BusinessTravel_Travel_Frequently : 2 valeurs uniques
- BusinessTravel_Travel_Rarely : 2 valeurs uniques
- Educat

In [22]:
display(company_df.head())

,Age,Attrition,DistanceFromHome,Education,EmployeeCount,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,Over18,...,JobRole_Sales Executive,JobRole_Sales Representative,Department_Research & Development,Department_Sales,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Total_Hours
0,51,0,6,1,1,1,1,131160,1.0,1,...,0.0,0.0,0.0,1.0,3.0,4.0,2.0,3,3,1710.69
1,31,1,10,0,1,2,1,41890,0.0,1,...,0.0,0.0,1.0,0.0,3.0,2.0,4.0,2,4,1821.68
2,32,0,17,3,1,3,4,193280,1.0,1,...,1.0,0.0,1.0,0.0,2.0,2.0,1.0,3,3,1697.20
3,38,0,2,4,1,4,3,83210,3.0,1,...,0.0,0.0,1.0,0.0,4.0,4.0,3.0,2,3,1690.51
4,32,0,10,0,1,5,1,23420,4.0,1,...,1.0,0.0,1.0,0.0,4.0,1.0,3.0,3,3,1961.51


---

### 5. <a id='missing-check'>Gestion des valeurs manquantes</a>

In [23]:
if (company_df.isnull().sum() > 0).any():
    missing_values = company_df.isnull().sum()[company_df.isnull().sum() > 0]
    missing_percentage = (missing_values / len(company_df) * 100).round(2)
    print("Valeurs manquantes par colonne :")
    for col in missing_values.index:
        print(f"{col} : {missing_values[col]} valeurs ({missing_percentage[col]}%)")
else:
    print("Il n'y a pas de valeurs manquantes dans le dataset")

Il n'y a pas de valeurs manquantes dans le dataset


---

### 6. <a id='doublons'>Gestion des valeurs dupliquées</a>

In [24]:
if company_df.duplicated().sum() > 0:
    print(f"Nombre de valeurs dupliquées : {company_df.duplicated().sum()}")
    merged_dataset = company_df.drop_duplicates()
    print(f"Nombre de lignes après suppression des doublons : {merged_dataset.shape[0]}")
else:
    print("Il n'y a pas de valeurs dupliquées dans le dataset")

Il n'y a pas de valeurs dupliquées dans le dataset


---

### 7. <a id='types'>Vérification des types de données</a>

In [25]:
numerical_columns = merged_dataset.select_dtypes(include=['int64', 'float64']).columns

for column in numerical_columns:
    invalid_values = merged_dataset[column].apply(lambda x: isinstance(x, (str)))
    if invalid_values.any():
        print(f"La colonne {column} contient des valeurs non numériques :")
        print(merged_dataset[invalid_values][column])
        print(DELIMITER)

if not invalid_values.any():
    print("Aucune valeur non numérique a été trouvée dans les colonnes numériques")

Aucune valeur non numérique a été trouvée dans les colonnes numériques


In [26]:
categorical_columns = merged_dataset.select_dtypes(include=['object']).columns

for column in categorical_columns:
    print(f"Vérification des valeurs uniques dans la colonne {column}")
    print(merged_dataset[column].unique())
    print(DELIMITER)

---

### 8. <a id='pipeline'>Pipeline de traitement des données</a>

# <font color="red"><center>**WIP**</center></font>